# 2. Customer Order Processing

Scenario: You are building a customer order processing system for an e-commerce company. The system needs to interact with a MySQL database to store customer orders, products, and order details

1. Design a MySQL database schema for the order processing system, including tables for customers, products, and orders

2. Write a Python program that connects to the database and allows customers to place new orders

3. Implement a feature that calculates the total cost of an order and updates product quantities in the database

4. How would you handle cases where a product is no longer available when a customer places an order?

5. Develop a function to generate order reports for the company's finance department

In [7]:
import mysql.connector
db = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="Achu@1821101",
    database="Customer_Order")
print(db)

In [8]:
cursor = db.cursor()

In [9]:
cursor

In [ ]:
cursor.execute("CREATE TABLE Customers(customer_id INT AUTO_INCREMENT PRIMARY KEY,first_name VARCHAR(255) NOT NULL,last_name VARCHAR(255) NOT NULL,email VARCHAR(255) NOT NULL)")
cursor.execute("CREATE TABLE Products(product_id INT AUTO_INCREMENT PRIMARY KEY,product_name VARCHAR(255) NOT NULL,price DECIMAL(10, 2) NOT NULL,quantity INT NOT NULL)")
cursor.execute("CREATE TABLE Orders(order_id INT AUTO_INCREMENT PRIMARY KEY,customer_id INT NOT NULL,order_date DATE NOT NULL,FOREIGN KEY (customer_id) REFERENCES Customers(customer_id))")

In [10]:
cursor.execute("alter table Orders add column total_cost INT NOT NULL ")

In [ ]:
cursor.execute("alter table Products add column quantity_available INT NOT NULL ")

In [4]:
def place_order(customer_id, product_id, quantity):
    # Check if the product is available
    cursor.execute("SELECT quantity_available FROM Products WHERE product_id = %s", (product_id,))
    available_quantity = 1

    if available_quantity < quantity:
        print("Product not available in the desired quantity.")
        return

    # Calculate the total cost
    cursor.execute("SELECT price FROM Products WHERE product_id = %s", (product_id,))
    unit_price = cursor.fetchone()[0]
    total_cost = unit_price * quantity
    cursor.execute("INSERT INTO Orders (customer_id, total_cost) VALUES (%s, %s)", (customer_id, total_cost))
    order_id = cursor.lastrowid
    cursor.execute("INSERT INTO OrderDetails (order_id, product_id, quantity, unit_price) VALUES (%s, %s, %s, %s)",(order_id, product_id, quantity, unit_price))
    new_quantity = available_quantity - quantity
    cursor.execute("UPDATE Products SET quantity_available = %s WHERE product_id = %s", (new_quantity, product_id))
    db.commit()
    print("Order placed successfully.")
place_order(1, 101, 3)  


Product not available in the desired quantity.


In [12]:
def generate_order_report(start_date, end_date):
    cursor.execute("SELECT Customers.first_name, Customers.last_name, Orders.order_date, Orders.total_cost "
                   "FROM Customers "
                   "JOIN Orders ON Customers.customer_id = Orders.customer_id "
                   "WHERE Orders.order_date BETWEEN %s AND %s", (start_date, end_date))
    
    orders = cursor.fetchall()

    if not orders:
        print("No orders found for the specified date range.")
        return

    print("Order Report:")
    print("Customer Name\tOrder Date\tTotal Cost")
    for order in orders:
        print(f"{order[0]} {order[1]}\t{order[2]}\t${order[3]:.2f}")
generate_order_report("2023-01-01", "2023-12-31")

No orders found for the specified date range.
